In [5]:
import sys
from datetime import datetime


import ipywidgets as widgets
import pydantic
from rich import print
from ipywidgets import GridBox, Layout

from sweets.core import Workflow

input_list_bbox = [
    widgets.BoundedFloatText(description="Left (lon.)", min=-180, max=180),
    widgets.BoundedFloatText(description="Bottom (lat.)", min=-90, max=90),
    widgets.BoundedFloatText(description="Right (lon.)", min=-180, max=180),
    widgets.BoundedFloatText(description="Top (lat.)", min=-90, max=90),
]

# Make a 3x3 grid, where the left/bottom/right/top are in
#  in positions (1, 0), (2, 1), (1, 2), (0, 1)
filler = widgets.Label("")
panel_bbox = GridBox(
    children=[
        filler,
        input_list_bbox[3],
        filler,
        input_list_bbox[0],
        filler,
        input_list_bbox[2],
        filler,
        input_list_bbox[1],
        filler,
    ],
    layout=Layout(
        width="100%",
        grid_template_rows="auto auto auto",
        grid_template_columns="33% 33% 33%",
        grid_template_areas="""
        ". top ."
        "left . right "
        ". bottom ."
        """,
    ),
)

input_wkt = widgets.Text(placeholder="POLYGON ( ... )")
bbox_options = widgets.VBox(
    [
        widgets.HTML("<h2>AOI (enter either bbox or WKT)</h2>"),
        widgets.Tab(
            [
                panel_bbox,
                input_wkt,
            ],
            titles=("Lon/lat bounding box (degrees)", "WKT string"),
        ),
    ]
)
input_start = widgets.DatePicker(description="Start")
input_end = widgets.DatePicker(description="End", value=datetime.today())
input_track = widgets.BoundedIntText(
    value=1,
    min=1,
    max=175,
    step=1,
    description="Track:",
)
panel_data_options = widgets.VBox(
    [
        widgets.HTML("<h4>(Required) Date range and relative orbit</h4>"),
        input_start,
        input_end,
        input_track,
    ]
)


input_work_dir = widgets.Text(placeholder=".")
panel_work_dir = widgets.VBox(
    [
        widgets.HTML("Working directory to store results"),
        input_work_dir,
    ]
)

input_orbit_dir = widgets.Text(placeholder="~/existing_orbit_dir")
panel_orbit_dir = widgets.VBox(
    [
        widgets.HTML("Directory containing orbit files (optional)"),
        input_orbit_dir,
    ]
)
input_data_dir = widgets.Text(placeholder="~/existing_data_dir")
panel_data_dir = widgets.VBox(
    [
        widgets.HTML("Directory containing existing Sentinel-1 data (optional)"),
        input_data_dir,
    ]
)


input_pick_baseline_bandwidth = widgets.Dropdown(
    options=["Temporal baseline", "Bandwidth"],
    value="Bandwidth",
    description="Ifg. Method",
)
input_ifg_max_bandwidth = widgets.BoundedIntText(value=3, min=1)
input_ifg_max_temp_baseline = widgets.BoundedIntText(value=90, min=1)
# Link the choice from the dropdown to the appropriate input
# Make sure the other is disabled
dl1 = widgets.dlink(
    (input_pick_baseline_bandwidth, "value"),
    (input_ifg_max_bandwidth, "disabled"),
    lambda x: x != "Bandwidth",
)
dl2 = widgets.dlink(
    (input_pick_baseline_bandwidth, "value"),
    (input_ifg_max_temp_baseline, "disabled"),
    lambda x: x != "Temporal baseline",
)
input_list_looks = [
    widgets.BoundedIntText(description="az. looks", value=6, min=1),
    widgets.BoundedIntText(description="range looks", value=12, min=1),
]
looks_options = widgets.HBox(input_list_looks)


ifg_options = widgets.VBox(
    [
        widgets.HTML("<h2>Interferogram selection options</h2>"),
        input_pick_baseline_bandwidth,
        widgets.Tab(
            [
                input_ifg_max_bandwidth,
                input_ifg_max_temp_baseline,
            ],
            titles=("Temporal baseline", "Bandwidth (nearest-n ifgs)"),
        ),
        looks_options,
    ]
)

input_n_workers = widgets.BoundedIntText(value=4, min=1)
input_tpw = widgets.BoundedIntText(value=8, min=1)
worker_options = widgets.VBox(
    [
        widgets.HTML("<h2>Worker options</h2>"),
        widgets.Tab(
            [input_n_workers, input_tpw],
            titles=("Number of workers", "Threads per worker"),
        ),
    ]
)


def _get_widget_values():
    # Connect the inputs to the workflow keyword args
    kwargs = {
        "start": input_start.value,
        "end": input_end.value,
        "bbox": [inp.value for inp in input_list_bbox],
        "wkt": input_wkt.value,
        "track": input_track.value,
        "looks": [inp.value for inp in input_list_looks],
        "n_workers": input_n_workers.value,
        "threads_per_worker": input_tpw.value,
    }
    if not input_ifg_max_temp_baseline.disabled:
        kwargs["max_temporal_baseline"] = input_ifg_max_temp_baseline.value
    else:
        kwargs["max_bandwidth"] = input_ifg_max_bandwidth.value
    if input_orbit_dir.value:
        kwargs["orbit_dir"] = input_orbit_dir.value
    if input_data_dir.value:
        kwargs["data_dir"] = input_data_dir.value
    if input_work_dir.value:
        kwargs["work_dir"] = input_data_dir.value
    return kwargs


save_button = widgets.Button(description="Save config", button_style="info")
output = widgets.Output()


def on_save_button_clicked(b):
    output.clear_output()
    workflow_kwargs = _get_widget_values()
    try:
        w = Workflow(start_dask=False, **workflow_kwargs)
        config_file = w.work_dir / "sweets_config.yaml"
        with output:
            print(f"Saving config to {config_file}:")
            w.to_yaml(output_path=sys.stdout)
            w.save(config_file=config_file)
    except pydantic.ValidationError as e:
        with output:
            print(e)
        return


save_button.on_click(on_save_button_clicked)

run_button = widgets.Button(description="Run workflow", button_style="danger")


def on_run_button_clicked(b):
    output.clear_output()

    workflow_kwargs = _get_widget_values()
    try:
        w = Workflow(**workflow_kwargs)
    except pydantic.ValidationError as e:
        with output:
            print(e)
        return
    with output:
        print("Running workflow!")
        w.run()


run_button.on_click(on_run_button_clicked)
buttons = widgets.HBox([save_button, run_button])

# APP: https://ipywidgets.readthedocs.io/en/stable/examples/Layout%20Templates.html
app = widgets.VBox(
    [
        bbox_options,
        panel_data_options,
        panel_work_dir,
        panel_orbit_dir,
        panel_data_dir,
        ifg_options,
        worker_options,
        buttons,
        output,
    ]
)
app